# PRS-preselection-based trans QTL analysis

# Summary

This is a short summary of our polygenic risk score (PRS) based pathway enrichment approach to preselect candidate genes for trans QTL analyses.

You can find a preprint version of the manuscript  
Ines Assum & Julia Krause et al., Tissue-specific multiOMICs analysis of atrial fibrillation, *bioRxiv* (2020)  
here:
https://doi.org/10.1101/2020.04.06.021527

We address a key hypothesis about the existence of core genes as postulated in the omnigenic model by [Liu et al.](https://doi.org/10.1016/j.cell.2019.04.014), *Cell* (2019). Core genes are central genes with trans-associations to GWAS loci, whose expression levels directly affect a disease phenotype. Here we sought to identify candidate core genes for AF to understand the contribution of trans-genetic effects in the pathology of AF. To prioritize genes satisfying the properties predicted by the omnigenic model, we evaluated the accumulation of trans-effects, their relevance in gene regulatory networks, and the disease association by the following strategy:


i) Evaluate cumulated trans-effects of disease-associated variants on expression by ranking genes based on their correlation of mRNA and protein abundance with a PRS (eQTS) as proposed by [Võsa et al.](https://doi.org/10.1101/447367), *bioRxiv* (2018)

ii) Identify genes sharing molecular function and representing biological networks that propagate genetic trans-effects to core genes by pathway enrichment analysis (GSEA) on the eQTS rankings. Genes driving the enrichment of multiple gene sets were selected as core gene candidates.

iii) Establish the link between the core gene candidates and the disease based on a significant trans eQTL GWAS hit.

# Example

## Requirements

This tutorial uses the following packages:

* [fgsea](https://bioconductor.org/packages/release/bioc/html/fgsea.html)
* [MatrixEQTL](https://github.com/andreyshabalin/MatrixEQTL)
* [eQTLpipeline](https://github.com/matthiasheinig/eQTLpipeline)

and data

* [1k_genomes_PRS_expr.tsv](https://github.com/heiniglab/symatrial/blob/master/example_data/1k_genomes_PRS_expr.tsv)
* [1k_genomes_AF_snps.txt](https://github.com/heiniglab/symatrial/blob/master/example_data/1k_genomes_AF_snps.txt)
* [GO bp .gmt file](https://www.gsea-msigdb.org/gsea/msigdb/download_file.jsp?filePath=/msigdb/release/7.1/c5.bp.v7.1.symbols.gmt).

Packages are installed in the first code chunk (running it might take up to 15 minutes). You can skip the first code chunk and only load the precomputed results, if you are in a hurry.

All necessary data is downloaded automatically, you only need to download gene set annotations for the GO biological processes ([GO bp .gmt file](https://www.gsea-msigdb.org/gsea/msigdb/download_file.jsp?filePath=/msigdb/release/7.1/c5.bp.v7.1.symbols.gmt)).
To temporarily upload files to this runtime, click on the folder symbol on the top left. Using the panel that opens, you can choose "Upload" to open a window to browse your local file system.

In [0]:
#Install necessary libraries
install.packages(c("devtools","RColorBrewer"), dependencies = T, clean = T)
BiocManager::install("fgsea", dependencies = T, clean = T)

library(devtools)
devtools::install_github("andreyshabalin/MatrixEQTL", force=T)
devtools::install_github("matthiasheinig/eQTLpipeline", force=T)

#Load libraries
library(fgsea)
library(MatrixEQTL)
library(eQTLpipeline)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘clisymbols’, ‘gargle’, ‘cyclocomp’, ‘xmlparsedata’, ‘highlight’, ‘parsedate’, ‘rappdirs’, ‘whoami’, ‘hunspell’, ‘BiocManager’, ‘curl’, ‘foghorn’, ‘gmailr’, ‘lintr’, ‘mockery’, ‘pingr’, ‘pkgdown’, ‘rhub’, ‘spelling’




## Loading data

In the following, we show the general procedure based on public data (272 individuals from the 1000 genomes LCL data).  
We supply a small test dataset with pre-computed PRS values for AF and CAD as well as gene expression for 830 genes (`1k_genomes_PRS_expr.tsv`). Additionally, genotypes for 109 pruned AF SNPs for those individuals are supplied (`1k_genomes_AF_snps.txt`) in MatrixEQTL format.

In [0]:
df <- read.csv(file = "https://github.com/heiniglab/symatrial/raw/master/example_data/1k_genomes_PRS_expr.tsv",
               sep = "\t", h = T, stringsAsFactors = F)
print.data.frame(df[1:5, 1:10],
                 row.names = F, digits = 3)

## eQTS ranking
First step is to calculate the eQTS rankings:

In [0]:
eQTS <- data.frame(matrix(nrow = 830, ncol = 5),
                   stringsAsFactors = F,
                   row.names = colnames(df)[-c(1:7)])
colnames(eQTS) <- c("id", "Estimate", "StdError", "tvalue", "pvalueT")
data <- df[, c("id", "PRS.AF", "PRS.AF.percentile",
               "PRS.CAD", "PRS.CAD.percentile",
               "sex", "population")]
for (i in 8:(dim(df)[2])){
    id <- colnames(df)[i]
    data$expr <- df[, id]
    lmres <- lm(expr ~ sex + PRS.AF.percentile,
                data = data)
    eQTS[id, ] <- c(id,
                    summary(lmres)$coefficients["PRS.AF.percentile", ])
    if(i %% 100 == 0) print(paste0("Gene ", i-7, " / ", dim(df)[2]-7, " done"))
}
eQTS[, -1] <- apply(eQTS[, -1], 2, as.numeric)
eQTS <- eQTS[order(eQTS$pvalueT), ]

print("Top five associated genes:")
print.data.frame(eQTS[1:5, ],
                 row.names = F, digits = 3)

## Load pathway annotations

As mentioned before, please download GO biological processes gene sets first, [GO bp .gmt file](https://www.gsea-msigdb.org/gsea/msigdb/download_file.jsp?filePath=/msigdb/release/7.1/c5.bp.v7.1.symbols.gmt),
then import them into R using the fgsea package.

In [0]:
if(!file.exists("c5.bp.v7.1.symbols.gmt")){
  cat("To actually run the tutorial, please install the package 'fgsea' and 
  download pathway annotations to move on! 
  To upload the .gmt file, click on the folder symbol on the top left. 
  Using the panel that opens, you can choose 'Upload' to open a window to 
  browse your local file system.")
} else {
  try({
    gmt <- gmtPathways("c5.bp.v7.1.symbols.gmt")
    print("GO biological processes gene set annotations loaded.")
    head(gmt, 1)
  }, print("Loading gene set annotations not successful. Library 'fgsea' probably not installed."))
}

## Compute gene set enrichment analysis on eQTS ranking:


In [0]:
fgsea.res <- NULL
try({
  set.seed(1111)
  rank <- eQTS[, "tvalue"]
  names(rank) <- eQTS[, "id"]
  fgsea.res <- fgsea(gmt,
                     rank,
                     nperm=10000,
                     minSize = 5,
                     maxSize=100)
  fgsea.res <- fgsea.res[order(fgsea.res$pval), ]
})
  
if(is.null(fgsea.res)){
  print("'fgsea' is not installed or pathway annotations are missing. Let's load the results ...")
  download.file("https://github.com/heiniglab/symatrial/raw/master/example_data/1k_genomes_fgsea_eQTS.RDS",
                dest = "1k_genomes_fgsea_eQTS.RDS")
  fgsea.res <- data.frame(readRDS("1k_genomes_fgsea_eQTS.RDS"))
  print("... fgsea results loaded.")
}

Top five pathways:

In [0]:
head(fgsea.res[1:5, ])

## Extract the leadingEdge as candidate genes:

As this is only an example dataset for demonstration, we choose a significane threshold of P < 0.01. However, for a real analysis we would recommend a much more stringent cutoff based on the adjusted p-value (FDR).

For each gene set, the leadingEdge contains the genes that possibly drive the enrichment. We therefore use those genes for significant pathways. To narrow it down further, we check if the same genes are contained in the leadingEdge of multiple pathways.  
We take all genes, that appear more than 5 times (8 genes in total) as candidates for the trans eQTL analysis.

In [0]:
lead <- data.frame(table(unlist( fgsea.res[fgsea.res$pval<0.01, "leadingEdge"])))
hist(lead$Freq, breaks = 10,
      main = "Distribution: Frequencies of genes in multiple leading edges:")

Let's select the top candidates that appear more than 5 times:

In [0]:
candidates <- as.character(lead[lead$Freq>5, "Var1"])
print(candidates)

## Run trans eQTL analysis

Let's run a trans eQTL analysis for those eight genes and the 109 AF GWAS SNPs:


In [0]:
eQTL <- NULL
try({
  eQTL <- trans.qtl(prefix = "/tmp/example",
                    genotype_file_name = "https://github.com/heiniglab/symatrial/raw/master/example_data/1k_genomes_AF_snps.txt",
                    expression_file_name = t(df[, candidates]),
                    covariates_file_name = df$sex,
                    threshold = 1,
                    compute.all = T,
                    min.pv.by.genesnp = T,
                    save.memory = F,
                    load.qtls = T)
})
if(is.null(eQTL)){
  print("Libraries 'MatrixEQTL' and 'eQTLpipeline' not available. Let's load the results...")
  download.file("https://github.com/heiniglab/symatrial/raw/master/example_data/1k_genomes_trans_eQTLs.RDS",
                dest = "1k_genomes_trans_eQTLs.RDS")
  eQTL <- readRDS("1k_genomes_trans_eQTLs.RDS")
  print("... eQTL results loaded.")
}

Top trans eQTLs:

In [0]:
head(eQTL$all$eqtls[order(eQTL$all$eqtls$pvalue), ])